In [1]:
import pickle
import gensim
from gensim.models.word2vec import Word2Vec
import wget
import gensim.downloader as api
from gensim.models import KeyedVectors


In [2]:
with open("text_lemmentized.txt", "rb") as fp:
    text_lemmentized = pickle.load(fp)

In [3]:
text_lemmentized

[['across',
  'north',
  'america',
  'planners',
  'downtown',
  'managers',
  'health',
  'experts',
  'working',
  'tirelessly',
  'reinvent',
  'public',
  'space',
  'social',
  'distancing',
  'light',
  'covid-19',
  'pandemic'],
 ['pop-up',
  'bike',
  'lanes',
  'example',
  'aim',
  'give',
  'people',
  'safe',
  'alternative',
  'public',
  'transit'],
 ['pedestrianization',
  'projects',
  'city',
  'areas',
  'downtown',
  'streets',
  'give',
  'people',
  'space',
  'shop',
  'support',
  'favorite',
  'local',
  'businesses'],
 ['large',
  'roads',
  'vancouvers',
  'stanley',
  'park',
  'closed',
  'give',
  'options',
  'people',
  'exercise',
  'difficult',
  'time'],
 ['pedestrianization',
  'consists',
  'converting',
  'street',
  'area',
  'pedestrian-only',
  'use'],
 ['usually',
  'aims',
  'provide',
  'better',
  'accessibility',
  'mobilityfor',
  'pedestrians',
  'enhance',
  'amount',
  'shopping',
  'business',
  'activities',
  'area',
  'and/or',
  'i

In [4]:
my_q = 300 
mcount = 5  

w2v = Word2Vec(size = my_q, min_count = mcount)
w2v.build_vocab(text_lemmentized)

In [28]:
# Download pre-trained word2vec
#googlenews = wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
#wv = api.load('word2vec-google-news-300')
#wv.save("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
#w2v.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)
# w = models.KeyedVectors.load_word2vec_format(
#     '../GoogleNews-vectors-negative300.bin', binary=True)
w2v = KeyedVectors.load('word2vec-google-news-300', mmap='r')


In [11]:
w2v['city'].shape

(300,)

In [9]:
def my_vector_getter(word, wv) :
    try:
        word_array = wv[word].reshape(1,-1)
        return word_array
    except KeyError:
        print('word: <', word, '> not in vocabulary!')

In [12]:
my_vector_getter('city', w2v).shape

(1, 300)

In [13]:
w2v.similar_by_word('city')

KeyboardInterrupt: 